In [ ]:
from IPython.display import display
from copy import deepcopy
import numpy as np
import pandas as pd
import pyvinecopulib as pv
from gcimpute.helper_mask import mask_MCAR
from vcimpute import simulator, diagonalize

# d = 4
# mat = np.empty(shape=(d,d),dtype=object)
# pcs = make_triangular_array(d)
# pv.Vinecop(mat, pcs)
# test if this selection works

In [ ]:
def make_triangular_array(d):
    pair_copulas = np.empty(shape=(d-1,), dtype='object')
    for j in range(d-1)[::-1]:
        pair_copulas[j] = list(np.empty(shape=(d-j-1,), dtype='object'))
    return list(pair_copulas)

In [ ]:
n = 1000
d = 4
p = 0.2

structure = pv.RVineStructure.simulate(d)

pair_copulas = []
for j in range(d - 1):
    tmp = []
    pair_copulas.append(tmp)
    for _ in range(d - j - 1):
        rho = np.minimum(np.maximum(np.random.beta(1, 0.75), 0.01), 0.99)
        tmp.append(pv.Bicop(family=pv.BicopFamily.gaussian, parameters=[[rho]]))

cop1 = pv.Vinecop(structure, pair_copulas)

U = cop1.simulate(n=n, seeds=list(1 + np.arange(d)))

In [ ]:
U_mask = mask_MCAR(U, p)

In [ ]:
### Impute

In [ ]:
vcop_controls = pv.FitControlsVinecop(family_set=[pv.BicopFamily.gaussian])
cop = pv.Vinecop(data=U_mask, controls=vcop_controls)
cop.matrix
display(np.count_nonzero(np.isnan(U_mask)))

In [ ]:
U_impute = U_mask.copy()
missing1 = np.isnan(U_impute[:,0])
missing2 = np.isnan(U_impute[:,1])
missing3 = np.isnan(U_impute[:,2])
missing4 = np.isnan(U_impute[:,3])

In [ ]:
# impute ord0
candidate2 = simulator.simulate_order0(cop, U_impute)[missing2]
U_impute[missing2,1] = candidate2
display(np.count_nonzero(np.isnan(U_impute)))

In [ ]:
cop = pv.Vinecop(data=U_impute, controls=vcop_controls)

In [ ]:
# diagonlize, impute (old) ord1
cop_tmp = diagonalize.diagonalize_copula(cop, 4)
candidate4 = simulator.simulate_order0(cop_tmp, U_impute)[missing4]
U_impute[missing4, 3] = candidate4
display(np.count_nonzero(np.isnan(U_impute)))

In [ ]:
# fit subvine
cop.matrix

In [ ]:
np.count_nonzero(np.isnan(U_impute), axis=0)